In [1]:

import pandas as pd
import numpy as np

from sklearn.metrics import SCORERS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

from sklearn.model_selection import train_test_split

from imblearn.under_sampling import RandomUnderSampler

from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import svm

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import MaxAbsScaler
from sklearn.kernel_approximation import RBFSampler

from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.ensemble import AdaBoostClassifier

In [79]:
tf = pd.read_csv('../featuresMagui.csv') #mis features 
tf_1 = pd.read_csv('../features_seba.csv') #los features de Seba
tf_2 = pd.read_csv('../santi_timefeatures.csv')
tf_3 = pd.read_csv('../Santi_FeaturesConEventos.csv')
tf_4 = pd.read_csv('../featuresMagui2.csv')
tf_5 = pd.read_csv('../fetures_nuevos_santi.csv')
labels = pd.read_csv('../../data/labels_training_set.csv') #las personas de las cuales tengo Info
personas =pd.read_csv('../../data/trocafone_kaggle_test.csv') #las personas a las que le tengo que predecir

In [80]:
#Datos a entregar
datos = pd.merge(tf, tf_1, on = 'person', how = 'inner')
datos = pd.merge(datos, tf_2, on = 'person', how = 'inner')
datos = pd.merge(datos, tf_3, on = 'person', how = 'inner')
datos = pd.merge(datos, tf_4, on = 'person', how = 'inner')
datos = pd.merge(datos, tf_5, on = 'person', how = 'inner')
datos = pd.merge(personas, datos, on = 'person', how = 'inner')

Cargo los features de los csv

In [81]:
labels_seba = pd.read_csv('../../data/cuarenta_cols.csv')

In [82]:
labels = pd.read_csv('../../data/labels_training_set.csv')

In [83]:
labels_seba = pd.merge(labels, labels_seba, on = 'person', how = 'inner')

In [84]:
labels_seba =labels_seba.drop('person',axis = 1)

In [85]:
atributos = labels_seba.columns.tolist()

In [86]:

atributos.remove('label')

In [87]:
y = labels_seba['label'].ravel()
X = labels_seba.loc[:,atributos]

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 123)

In [89]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf_1 = ExtraTreesClassifier()
clf_2 = AdaBoostClassifier()
clf_3  = GradientBoostingClassifier()
clf_4 = XGBClassifier(learning_rate =0.05,n_estimators=1000,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)
clasificadores = [(clf,False), (clf_1,False), (clf_2,False), (clf_3,False), (clf_4,True)]

In [90]:
for clasificador in clasificadores: 
    if(clasificador[1]):
        clasificador[0].fit(X_train, y_train, early_stopping_rounds = 5, eval_set=[(X_test, y_test)])
    else:
        clasificador[0].fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0]	validation_0-error:0.052008
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.049434
[2]	validation_0-error:0.048404
[3]	validation_0-error:0.047374
[4]	validation_0-error:0.047374
[5]	validation_0-error:0.047889
[6]	validation_0-error:0.046859
[7]	validation_0-error:0.046859
[8]	validation_0-error:0.047374
[9]	validation_0-error:0.048919
[10]	validation_0-error:0.048919
[11]	validation_0-error:0.048404
Stopping. Best iteration:
[6]	validation_0-error:0.046859



In [91]:
importanciaPorFeature = []
features = X.columns.tolist()
for clasificador in clasificadores:        
    importanciaPorFeature.append(clasificador[0].feature_importances_) 

In [92]:
importancia = {}
x = 0
for i in range(5):
    for feature in importanciaPorFeature[i]:
        importancia[features[x]] = importancia.get(features[x], 0)
        importancia[features[x]]+= feature
        x+=1
    x=0

In [93]:
for importancia_feat in importancia.keys():
    importancia[importancia_feat] /= 5

In [94]:
[importancia_feat,importancia[importancia_feat]

SyntaxError: unexpected EOF while parsing (<ipython-input-94-d2eee00b72b3>, line 1)

In [95]:
atributos_importantes = []
for importancia_feat in importancia.keys():
    if importancia[importancia_feat] > 0.01:
        atributos_importantes.append(importancia_feat)

In [96]:
len(atributos_importantes)

35

In [97]:
#atributos_importantes

In [98]:
atributos_importantes.append('label')
atributos_importantes.append('person')
#atributos_label = atributos_importantes.append('person')
labels_f_filtrado= labels_seba.loc[:,atributos_importantes]

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [99]:
labels_f_filtrado.head()

,brand listing,checkout,conversion,generic listing,search engine hit,searched products,staticpage,viewed product,visited site,vis_iPhone 7 Plus,...,dias_hasta_ultimo,viewed product mes 4,visited site mes 4,distan_dias,cant_dias_dist,modelos_dist,promedio_por_dia_x,promedio_por_mes,label,person
0,3.0,1.0,1.0,15.0,1.0,0.0,1.0,23.0,17.0,0.0,...,0,0.0,0.0,8,10,12,6.80,68.0,0,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3,0.0,0.0,0,1,0,2.00,2.0,0,NaN
2,14.0,1.0,0.0,9.0,4.0,6.0,0.0,31.0,22.0,0.0,...,2,8.0,11.0,108,12,7,8.00,24.0,0,NaN
3,5.0,0.0,0.0,3.0,1.0,0.0,0.0,24.0,4.0,0.0,...,4,0.0,0.0,8,4,11,9.25,37.0,0,NaN
4,0.0,2.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,0.0,...,12,0.0,0.0,0,2,1,8.50,17.0,0,NaN


In [100]:
labels_f_filtrado.to_csv('labels_seba_filtrado.csv', encoding='utf-8', index=False)

In [101]:
atributos_importantes.remove('label')


In [102]:
datos_filtrado = datos.loc[:,atributos_importantes]
datos_filtrado.head()

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,brand listing,checkout,conversion,generic listing,search engine hit,searched products,staticpage,viewed product,visited site,vis_iPhone 7 Plus,...,viewed product mes 5,dias_hasta_ultimo,viewed product mes 4,visited site mes 4,distan_dias,cant_dias_dist,modelos_dist,promedio_por_dia_x,promedio_por_mes,person
0,0.0,NaN,0.0,1.0,1.0,1.0,NaN,4.0,1.0,0.0,...,4.0,13,0.0,0.0,0,1,1,9.000000,9.0,4886f805
1,4.0,NaN,0.0,21.0,0.0,6.0,NaN,404.0,95.0,9.0,...,133.0,3,100.0,22.0,138,59,15,9.610169,113.4,0297fc1e
2,5.0,NaN,0.0,1.0,2.0,1.0,NaN,13.0,2.0,0.0,...,13.0,4,0.0,0.0,9,2,3,13.000000,26.0,2d681dd8
3,7.0,NaN,0.0,20.0,26.0,1.0,NaN,739.0,22.0,0.0,...,739.0,0,0.0,0.0,23,13,57,64.307692,836.0,cccea85e
4,8.0,NaN,0.0,14.0,13.0,9.0,NaN,177.0,20.0,0.0,...,177.0,9,0.0,0.0,4,5,9,51.400000,257.0,4c8a8b93


In [103]:
datos_filtrado.to_csv('datos_filtrado.csv', encoding='utf-8', index=False)